In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
from DataStructure import *

In [ ]:
def compute_smooth_gradient(height_map):
    """
    Compute gradient using simple finite differences (smooth result)
    """
    x, y = height_map.shape
    gradient = torch.zeros(x, y, 2, dtype=height_map.dtype, device=height_map.device)
    
    # Compute dx (gradient in x direction) - simple differences
    gradient[1:-1, :, 0] = (height_map[2:, :] - height_map[:-2, :]) / 2.0
    gradient[0, :, 0] = height_map[1, :] - height_map[0, :]
    gradient[-1, :, 0] = height_map[-1, :] - height_map[-2, :]
    
    # Compute dy (gradient in y direction) - simple differences
    gradient[:, 1:-1, 1] = (height_map[:, 2:] - height_map[:, :-2]) / 2.0
    gradient[:, 0, 1] = height_map[:, 1] - height_map[:, 0]
    gradient[:, -1, 1] = height_map[:, -1] - height_map[:, -2]
    
    return gradient

class PerlinNoise:
    def __init__(self, device='cpu', seed=None):
        self.device = device
        if seed is not None:
            torch.manual_seed(seed)
        # Generate permutation table (Ken Perlin's original)
        self.perm = torch.randperm(256, device=device)
        # Duplicate the permutation table to avoid wrapping
        self.perm = torch.cat([self.perm, self.perm])
    
    def fade(self, t):
        """Fade function: 6t^5 - 15t^4 + 10t^3"""
        return t * t * t * (t * (t * 6 - 15) + 10)
    
    def lerp(self, a, b, t):
        """Linear interpolation"""
        return a + t * (b - a)
    
    def grad(self, hash_val, x, y):
        """Gradient function for 2D"""
        h = hash_val & 3
        u = torch.where(h < 2, x, -x)
        v = torch.where(h == 0, y, torch.where(h == 1, -y, torch.where(h == 2, y, -y)))
        return u + v
    
    def noise2d(self, x, y):
        """Generate 2D Perlin noise"""
        # Find unit grid cell containing point
        X = x.floor().int() & 255
        Y = y.floor().int() & 255
        
        # Get relative coordinates within cell
        x = x - x.floor()
        y = y - y.floor()
        
        # Compute fade curves
        u = self.fade(x)
        v = self.fade(y)
        
        # Hash coordinates of 4 corners
        A = self.perm[X] + Y
        AA = self.perm[A]
        AB = self.perm[A + 1]
        B = self.perm[X + 1] + Y
        BA = self.perm[B]
        BB = self.perm[B + 1]
        
        # Compute gradients at corners
        grad_aa = self.grad(AA, x, y)
        grad_ba = self.grad(BA, x - 1, y)
        grad_ab = self.grad(AB, x, y - 1)
        grad_bb = self.grad(BB, x - 1, y - 1)
        
        # Interpolate
        lerp1 = self.lerp(grad_aa, grad_ba, u)
        lerp2 = self.lerp(grad_ab, grad_bb, u)
        
        return self.lerp(lerp1, lerp2, v)
    
    def generate_2d(self, width, height, scale=1.0, octaves=1, persistence=0.5, lacunarity=2.0, k=1):
        """Generate 2D Perlin noise texture"""
        # Create coordinate grids
        x = torch.linspace(0, width * scale, width, device=self.device)
        y = torch.linspace(0, height * scale, height, device=self.device)
        xx, yy = torch.meshgrid(x, y, indexing='ij')
        
        noises = torch.zeros_like(xx)
        if k:
            gradients = torch.zeros_like(xx)
        amplitude = 1.0
        frequency = 1.0
        max_value = 0.0
        
        # Generate octaves
        for _ in range(octaves):
            noise = self.noise2d(xx * frequency, yy * frequency) * amplitude
            if k:
                # Gradient trick from https://www.youtube.com/watch?v=gsJHzBTPG0Y
                gradient = compute_smooth_gradient(noise)
                magnitude = torch.sqrt(gradient[...,0].pow(2) + gradient[..., 1].pow(2))
                magnitude = (magnitude - magnitude.min()) / (magnitude.max() - magnitude.min())
                gradients += magnitude
                influence = 1 / (1 + k*gradients)
                noise = noise * influence

            noises += noise
            max_value += amplitude
            amplitude *= persistence
            frequency *= lacunarity
        
        # Normalize to [-1, 1]
        noises = noises / max_value
        
        return noises

# Simplified function interface
def perlin_noise_2d(width, height, scale=1.0, octaves=1, persistence=0.5, lacunarity=2.0, k=1, device='cpu', seed=42):
    perlin = PerlinNoise(device=device, seed=seed)
    return perlin.generate_2d(width, height, scale, octaves, persistence, lacunarity, k)

In [ ]:
def visualize_noise_2d(noise_tensor, cmap='terrain'):
    plt.figure(figsize=(10, 8))
    
    noise_np = noise_tensor.cpu().numpy()
    
    plt.imshow(noise_np, cmap=cmap, origin='lower')
    plt.colorbar(label='Noise Value')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

def normalize(x, mask=None):
    
    xMin = x.min() if mask is None else x[mask].min()
    xMax = x.max() if mask is None else x[mask].max()
    return (x - xMin) / (xMax - xMin)

def gen_info(pos, offsets):
    node = torch.floor(pos).int()
    indices = node[:, None] + offsets
    cellOffset = pos - node
    factor = torch.where(offsets.bool(), cellOffset[:, None], 1-cellOffset[:, None]).prod(-1)
    return node, indices, cellOffset, factor

def calculateHeight(heightMap, indices, factor):
    heights = heightMap[*indices.unbind(-1)]
    height = torch.sum(heights * factor, -1)
    return height

def calculateGradient(heightMap, indices, offset):
    heightNW, heightSW, heightNE, heightSE = heightMap[*indices.unbind(-1)].unbind(-1)
    gradientX = (heightNE - heightNW) * offset[..., 0] + (heightSE - heightSW) * (1 - offset[..., 0])
    gradientY = (heightSW - heightNW) * offset[..., 1] + (heightSE - heightNE) * (1 - offset[..., 1])
    gradient = torch.stack([gradientX, gradientY], -1)
    return gradient

def initialize(batchSize, mapSize, device, initialSpeed=1, initialWaterVolume=1):
    X = DataStructure()
    X.b.hw.pos = torch.rand(batchSize, 2).to(device) * (mapSize-1)
    X.b.hw.dir = torch.zeros(batchSize, 2).to(device)
    X.b.speed = torch.full((batchSize,), initialSpeed).to(device)
    X.b.water = torch.full((batchSize,), initialWaterVolume).to(device)
    X.b.sediment = torch.zeros(batchSize).to(device)
    X.b.iteration = torch.zeros(batchSize).to(device).int()
    return X.b

def gen_visual_map(X, seaLevel=0.3, riverCutoff=10):
    seaMask = X.h.w.height < seaLevel
    riverMask = X.h.w.river >= min(riverCutoff, 0.8*X.h.w.river.max().item())

    visualMap = normalize(X.h.w.height, mask=~seaMask) * 0.7 + 0.3
    visualMap[riverMask] =  normalize(X.h.w.height[riverMask]) * 0.2
    visualMap[seaMask] = 0
    return visualMap


In [ ]:
numIterations = 25000
mapSize = 2048
initialSpeed = 1
initialWaterVolume = 1
maxDropletLifetime = 30
inertia = .005
sedimentCapacityFactor = 4
minSedimentCapacity = .01
depositSpeed = 0.1
erodeSpeed = 0.1
gravity = 4
evaporateSpeed = .01
appendRate = 100
seaLevel = 0.3

batchSize = 1000000

X = DataStructure()

def gen_downhill_heightmap(mapSize):
    x = torch.sqrt((torch.arange(mapSize)).pow(2) + (torch.arange(mapSize) - mapSize/2).pow(2)[:,None])
    x = normalize(x)
    heightMap = perlin_noise_2d(mapSize, mapSize, initNoise=x, scale=0.01, octaves=6, persistence=0.5, seed=None)
    return normalize(heightMap)

def gen_twinpeak_heightmap(mapSize):
    r1 = 0.25
    r2 = 0.15
    x1 = torch.sqrt((torch.arange(mapSize) - (mapSize/2 - r1*mapSize)).pow(2) + (torch.arange(mapSize) - (mapSize/2 - r2*mapSize)).pow(2)[:,None])
    x2 = torch.sqrt((torch.arange(mapSize) - (mapSize/2 + r1*mapSize)).pow(2) + (torch.arange(mapSize) - (mapSize/2 + r2*mapSize)).pow(2)[:,None])
    x3 = torch.sqrt((torch.arange(mapSize) - (mapSize/2 + r1*mapSize)).pow(2) + (torch.arange(mapSize) - (mapSize/2 - r2*mapSize)).pow(2)[:,None])
    x0 = torch.sqrt((torch.arange(mapSize) - (mapSize/2 - r1*mapSize)).pow(2) + (torch.arange(mapSize) - (mapSize/2 + r2*mapSize)).pow(2)[:,None])
    edge = torch.min(torch.min((torch.arange(mapSize) - 0).pow(2), (torch.arange(mapSize) - mapSize - 1).pow(2)), torch.min((torch.arange(mapSize) - 0).pow(2)[:,None], (torch.arange(mapSize) - mapSize - 1).pow(2)[:,None]))
    x = normalize(-(normalize(x1).pow(0.5) + normalize(x2).pow(0.5) - 0.2*normalize(x0)))
    heightMap = perlin_noise_2d(mapSize, mapSize, scale=0.01, octaves=6, persistence=0.5, seed=None)
    return normalize(normalize(heightMap) + 2*x)

X.h.w.height = gen_twinpeak_heightmap(mapSize)

device = torch.device('cuda')
X.h.w.height = X.h.w.height.to(device)
X.h.w.river = torch.zeros_like(X.h.w.height).to(device)
X.b = initialize(batchSize, mapSize, device, initialSpeed=initialSpeed, initialWaterVolume=initialWaterVolume)

offsets = torch.tensor([[0,0], [0,1], [1,0], [1,1]]).to(device).int()
landscapes = []
rivers = []
for iteration in range(numIterations):
    X.b.hw.node, X.b.f.idx, X.b.hw.offset, X.b.f.factor = gen_info(X.b.hw.pos, offsets)

    X.b.height = calculateHeight(X.h.w.height, X.b.f.idx, X.b.f.factor)
    X.b.hw.gradient = calculateGradient(X.h.w.height, X.b.f.idx, X.b.hw.offset)

    X.b.hw.dir = X.b.hw.dir * inertia - X.b.hw.gradient * (1 - inertia)
    magnitude = X.b.hw.dir.pow(2).sum(-1, keepdim=True).clamp(1e-5).sqrt()
    X.b.hw.dir = X.b.hw.dir / magnitude
    X.b.hw.pos += X.b.hw.dir
    
    finish = ((X.b.hw.pos <= 0) | (X.b.hw.pos >= (mapSize-1))).any(-1) | (X.b.iteration >= maxDropletLifetime) | (X.h.w.height[X.b.hw.node.unbind(-1)] < (seaLevel-0.05))
    X.b.filter(~finish)

    _, newIdx, _, newFactor = gen_info(X.b.hw.pos, offsets)
    X.b.newHeight = calculateHeight(X.h.w.height, newIdx, newFactor)

    X.b.deltaHeight = X.b.newHeight - X.b.height
    X.b.sedimentCapacity = torch.clamp_max((-X.b.deltaHeight * X.b.speed * X.b.water * sedimentCapacityFactor), minSedimentCapacity)

    X.b.depositMask = (X.b.sediment > X.b.sedimentCapacity) | (X.b.deltaHeight > 0)
    amount = torch.zeros_like(X.b.sedimentCapacity)
    amount[X.b.depositMask] = torch.where(X.b.deltaHeight[X.b.depositMask] > 0, 
                                torch.min(X.b.deltaHeight[X.b.depositMask], X.b.sediment[X.b.depositMask]), 
                                (X.b.sediment[X.b.depositMask] - X.b.sedimentCapacity[X.b.depositMask]) * depositSpeed)
    amount[~X.b.depositMask] = torch.min((X.b.sedimentCapacity[~X.b.depositMask] - X.b.sediment[~X.b.depositMask]) * erodeSpeed, -X.b.deltaHeight[~X.b.depositMask])

    amount = amount[:, None] * X.b.f.factor
    height = X.h.w.height
    height[*X.b.f.idx.unbind(-1)] += torch.where(X.b.depositMask[:,None], amount, -amount)
    X.h.w.height = height

    amount = amount.sum(-1)
    X.b.sediment = X.b.sediment + torch.where(X.b.depositMask, -amount, amount)
    X.h.w.river[*X.b.f.idx.unbind(-1)] += (X.b.water[:, None] * X.b.f.factor)

    x = X.b.speed**2 + 2 * X.b.deltaHeight * gravity
    X.b.speed = torch.where(x > 0, x.sqrt(), 0)
    X.b.water = X.b.water * (1 - evaporateSpeed)
    X.b.iteration = X.b.iteration + 1
    
    toReplace = batchSize - len(X.b)
    if toReplace:
        new = initialize(toReplace, mapSize, device, initialSpeed=initialSpeed, initialWaterVolume=initialWaterVolume)
        X.b.cat(new, skip_errors=True, remove_missing=True)

    if iteration % appendRate == 0:
        landscapes.append(gen_visual_map(X, seaLevel=seaLevel).cpu().clone())
        rivers.append(torch.where(X.h.w.height < seaLevel, X.h.w.river.max(), X.h.w.river).cpu().clone())
        X.h.w.river = X.h.w.river * (1 - evaporateSpeed)**(appendRate)

    if iteration % 1000 == 0:
        visualize_noise_2d(gen_visual_map(X, seaLevel=seaLevel))
        visualize_noise_2d(torch.where(X.h.w.height < seaLevel, X.h.w.river.max(), X.h.w.river))

In [ ]:
import matplotlib.animation as animation

def animate_noise_2d(noise_tensors, title="Animated Perlin Noise", cmap='viridis', 
                    interval=100, save_path=None, figsize=(10, 8)):
    """
    Animate a sequence of 2D noise tensors over time
    
    Args:
        noise_tensors: List of torch.Tensor, each of shape [width, height]
        title: Title for the animation
        cmap: Colormap ('viridis', 'plasma', 'gray', 'terrain', 'coolwarm', etc.)
        interval: Time between frames in milliseconds
        save_path: Optional path to save animation (e.g., 'animation.gif' or 'animation.mp4')
        figsize: Figure size tuple
    
    Returns:
        matplotlib.animation.FuncAnimation object
    """
    # Convert all tensors to numpy arrays
    noise_arrays = [tensor.cpu().numpy() for tensor in noise_tensors]
    
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Find global min/max for consistent color scaling
    vmin = min(arr.min() for arr in noise_arrays)
    vmax = max(arr.max() for arr in noise_arrays)
    
    # Create initial image
    im = ax.imshow(noise_arrays[0], cmap=cmap, origin='lower', 
                   vmin=vmin, vmax=vmax, animated=True)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, label='Noise Value')
    
    # Set labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'{title} - Frame 0')
    
    def animate(frame):
        """Animation function called for each frame"""
        im.set_array(noise_arrays[frame])
        ax.set_title(f'{title} - Frame {frame}')
        return [im]
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, frames=len(noise_arrays),
                                 interval=interval, blit=True, repeat=True)
    
    # Save animation if path provided
    if save_path:
        if save_path.endswith('.gif'):
            anim.save(save_path, writer='pillow', fps=1000//interval)
        elif save_path.endswith('.mp4'):
            anim.save(save_path, writer='ffmpeg', fps=1000//interval)
        else:
            print(f"Unsupported format. Use .gif or .mp4")

# Function made by chatGPT and it's slow as shit, need to redo

anim = animate_noise_2d(landscapes, 
    title="Evolving Perlin Noise", 
    cmap='terrain',
    interval=20,  # 200ms between frames
    save_path='noise_animation.gif') 
    
anim = animate_noise_2d(rivers, 
    title="Evolving Perlin Noise", 
    cmap='terrain',
    interval=20,  # 200ms between frames
    save_path='rivers.gif') 

